<a href="https://colab.research.google.com/github/SARA3SAEED/Task2/blob/main/Task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python -m spacy download en_core_web_sm
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding
!pip install openai==0.27.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 65.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install gradio

In [ ]:
!pip install -U sentence-transformers rank_bm25

In [ ]:
import json
import pandas as pd
import time
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from collections import Counter
from heapq import nlargest
import nltk
import numpy as np
from tqdm import tqdm
from sentence_transformers import SentenceTransformer, util
# import tiktoken
from openai.embeddings_utils import get_embedding, cosine_similarity

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:
!pip install datasets pandas

In [ ]:
from datasets import load_dataset
import pandas as pd
ds= load_dataset('traversaal-ai-hackathon/hotel_datasets')
data = ds['train']
df = pd.DataFrame(data)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
df.head()

,hotel_name,hotel_description,review_title,review_text,rate,tripdate,hotel_url,hotel_image,price_range,rating_value,review_count,street_address,locality,country
0,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,"An exceptional boutique hotel, great value for...",None,NaN,February 2020,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,$ (Based on Average Nightly Rates for a Standa...,5.0,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye
1,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,You can’t get better than this.,None,NaN,March 2021,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,$ (Based on Average Nightly Rates for a Standa...,5.0,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye
2,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,Exceeds all expectations,None,NaN,March 2021,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,$ (Based on Average Nightly Rates for a Standa...,5.0,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye
3,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,"Great Location, Fantastic Accommodations",None,NaN,August 2021,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,$ (Based on Average Nightly Rates for a Standa...,5.0,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye
4,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,Perfection. It is all in the details.,None,NaN,June 2021,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,$ (Based on Average Nightly Rates for a Standa...,5.0,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye


In [ ]:
df.shape

(5997, 14)

In [ ]:
df.hotel_name.value_counts()

hotel_name
Romance Istanbul Hotel                      40
Motto by Hilton New York City Chelsea       40
The Bryant Park Hotel                       40
Pod Times Square                            40
Tempo by Hilton New York Times Square       40
                                            ..
San Francisco Marriott Fisherman's Wharf    40
Hilton San Francisco Financial District     40
The Donatello                               40
Hotel Campanile Paris Bercy Village         40
The FIDI Hotel                              37
Name: count, Length: 150, dtype: int64

In [ ]:
df=df.drop_duplicates()

In [ ]:
df.shape

(5997, 14)

In [ ]:
df.hotel_name.value_counts()

hotel_name
Romance Istanbul Hotel                      40
Motto by Hilton New York City Chelsea       40
The Bryant Park Hotel                       40
Pod Times Square                            40
Tempo by Hilton New York Times Square       40
                                            ..
San Francisco Marriott Fisherman's Wharf    40
Hilton San Francisco Financial District     40
The Donatello                               40
Hotel Campanile Paris Bercy Village         40
The FIDI Hotel                              37
Name: count, Length: 150, dtype: int64

In [ ]:
!mkdir /content/drive/MyDrive/Task2

mkdir: cannot create directory ‘/content/drive/MyDrive/Task2’: File exists


In [ ]:
df.to_csv('/content/drive/MyDrive/Task2/cities.csv',index=False)

In [ ]:
df.shape

(5997, 14)

In [ ]:
df.hotel_name.value_counts()

hotel_name
Romance Istanbul Hotel                      40
Motto by Hilton New York City Chelsea       40
The Bryant Park Hotel                       40
Pod Times Square                            40
Tempo by Hilton New York Times Square       40
                                            ..
San Francisco Marriott Fisherman's Wharf    40
Hilton San Francisco Financial District     40
The Donatello                               40
Hotel Campanile Paris Bercy Village         40
The FIDI Hotel                              37
Name: count, Length: 150, dtype: int64

In [ ]:
df.head()

,hotel_name,hotel_description,review_title,review_text,rate,tripdate,hotel_url,hotel_image,price_range,rating_value,review_count,street_address,locality,country
0,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,"An exceptional boutique hotel, great value for...",None,NaN,February 2020,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,$ (Based on Average Nightly Rates for a Standa...,5.0,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye
1,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,You can’t get better than this.,None,NaN,March 2021,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,$ (Based on Average Nightly Rates for a Standa...,5.0,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye
2,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,Exceeds all expectations,None,NaN,March 2021,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,$ (Based on Average Nightly Rates for a Standa...,5.0,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye
3,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,"Great Location, Fantastic Accommodations",None,NaN,August 2021,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,$ (Based on Average Nightly Rates for a Standa...,5.0,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye
4,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,Perfection. It is all in the details.,None,NaN,June 2021,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,$ (Based on Average Nightly Rates for a Standa...,5.0,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye


In [ ]:
sample = df[:10]
sample.head()

,hotel_name,hotel_description,review_title,review_text,rate,tripdate,hotel_url,hotel_image,price_range,rating_value,review_count,street_address,locality,country
0,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,"An exceptional boutique hotel, great value for...",None,NaN,February 2020,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,$ (Based on Average Nightly Rates for a Standa...,5.0,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye
1,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,You can’t get better than this.,None,NaN,March 2021,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,$ (Based on Average Nightly Rates for a Standa...,5.0,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye
2,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,Exceeds all expectations,None,NaN,March 2021,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,$ (Based on Average Nightly Rates for a Standa...,5.0,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye
3,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,"Great Location, Fantastic Accommodations",None,NaN,August 2021,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,$ (Based on Average Nightly Rates for a Standa...,5.0,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye
4,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,Perfection. It is all in the details.,None,NaN,June 2021,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,$ (Based on Average Nightly Rates for a Standa...,5.0,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye


In [ ]:
!pip install einops

In [ ]:
df.head()

,hotel_name,hotel_description,review_title,review_text,rate,tripdate,hotel_url,hotel_image,price_range,rating_value,review_count,street_address,locality,country
0,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,"An exceptional boutique hotel, great value for...",None,NaN,February 2020,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,$ (Based on Average Nightly Rates for a Standa...,5.0,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye
1,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,You can’t get better than this.,None,NaN,March 2021,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,$ (Based on Average Nightly Rates for a Standa...,5.0,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye
2,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,Exceeds all expectations,None,NaN,March 2021,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,$ (Based on Average Nightly Rates for a Standa...,5.0,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye
3,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,"Great Location, Fantastic Accommodations",None,NaN,August 2021,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,$ (Based on Average Nightly Rates for a Standa...,5.0,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye
4,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,Perfection. It is all in the details.,None,NaN,June 2021,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,$ (Based on Average Nightly Rates for a Standa...,5.0,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye


In [ ]:
from sentence_transformers import SentenceTransformer
from transformers import pipeline


model = SentenceTransformer("nomic-ai/nomic-embed-text-v1.5", trust_remote_code=True)
sentiment_model = pipeline('sentiment-analysis')
aspects = ["room", "service", "location", "staff", "food", "noise", "bed", "view"]
def extract_aspects(review, aspects):
    if review is None:
        return []
    sentences = review.split('. ')
    aspect_sentences = []
    for aspect in aspects:
        for sentence in sentences:
            if aspect in sentence.lower():
                aspect_sentences.append((aspect, sentence))
    return aspect_sentences
def analyze_sentiment(sentences):
    aspect_sentiments = {}
    for aspect, sentence in sentences:
        sentiment = sentiment_model(sentence)[0]
        aspect_sentiments[aspect] = {
            'sentence': sentence,
            'sentiment': sentiment['label'],
            'score': sentiment['score']
        }
    return aspect_sentiments
results = []
for review in sample["review_text"]:
    aspect_sentences = extract_aspects(review, aspects)
    aspect_sentiments = analyze_sentiment(aspect_sentences)
    results.append(aspect_sentiments)

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [ ]:
from sentence_transformers import SentenceTransformer
from transformers import pipeline
model = SentenceTransformer("nomic-ai/nomic-embed-text-v1.5", trust_remote_code=True)
sentiment_model = pipeline('sentiment-analysis')
aspects = ["room", "service", "location", "staff", "food", "noise", "bed", "view"]
def extract_aspects(review, aspects):
    if review is None:
        return []
    sentences = review.split('. ')
    aspect_sentences = []
    for aspect in aspects:
        for sentence in sentences:
            if aspect in sentence.lower():
                aspect_sentences.append((aspect, sentence))
    return aspect_sentences
def analyze_sentiment(sentences):
    aspect_sentiments = {}
    for aspect, sentence in sentences:
        sentiment = sentiment_model(sentence)[0]
        aspect_sentiments[aspect] = {
            'sentence': sentence,
            'sentiment': sentiment['label'],
            'score': sentiment['score']
        }
    return aspect_sentiments
results = []
for review in df["review_text"]:
    aspect_sentences = extract_aspects(review, aspects)
    aspect_sentiments = analyze_sentiment(aspect_sentences)
    results.append(aspect_sentiments)

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [ ]:
df['aspect_sentiments'] = results
df.head()

,hotel_name,hotel_description,review_title,review_text,rate,tripdate,hotel_url,hotel_image,price_range,rating_value,review_count,street_address,locality,country,aspect_sentiments
0,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,"An exceptional boutique hotel, great value for...",None,NaN,February 2020,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,$ (Based on Average Nightly Rates for a Standa...,5.0,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye,{}
1,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,You can’t get better than this.,None,NaN,March 2021,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,$ (Based on Average Nightly Rates for a Standa...,5.0,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye,{}
2,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,Exceeds all expectations,None,NaN,March 2021,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,$ (Based on Average Nightly Rates for a Standa...,5.0,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye,{}
3,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,"Great Location, Fantastic Accommodations",None,NaN,August 2021,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,$ (Based on Average Nightly Rates for a Standa...,5.0,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye,{}
4,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,Perfection. It is all in the details.,None,NaN,June 2021,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,$ (Based on Average Nightly Rates for a Standa...,5.0,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye,{}


In [ ]:
if review is not None:
        print(df['aspect_sentiments'])

0                                                      {}
1                                                      {}
2                                                      {}
3                                                      {}
4                                                      {}
                              ...                        
5992    {'room': {'sentence': 'I was asked to pay for ...
5993    {'room': {'sentence': 'The rooms were adequate...
5994    {'location': {'sentence': 'Stayed here for a f...
5995    {'room': {'sentence': 'The room was clean and ...
5996    {'room': {'sentence': 'The rooms are clean, ma...
Name: aspect_sentiments, Length: 5997, dtype: object


In [ ]:
df["reviews_search"] = (
    "Hotel_name: " + df.hotel_name.str.strip() +"; City: " + df.locality.str.strip()+"; review_text: " + df.review_text.str.strip()
)

In [ ]:
import re

df_copy = df.copy()

df_copy['reviews_search'] = df_copy['reviews_search'].apply(lambda x: re.sub('[^a-zA-z0-9\s]','',str(x)))

def lower_case(input_str):
    input_str = input_str.lower()
    return input_str

df_copy['reviews_search']= df_copy['reviews_search'].apply(lambda x: lower_case(x))

In [ ]:
if review is not None:
    sample = df_copy.sample(100)

In [ ]:
embedder = model

In [ ]:
sample.head()

,hotel_name,hotel_description,review_title,review_text,rate,tripdate,hotel_url,hotel_image,price_range,rating_value,review_count,street_address,locality,country,aspect_sentiments,reviews_search
1353,Hotel Emblem San Francisco,Check in and leave the status quo behind. Hote...,Wonderful Boutique Hotel!,None,NaN,November 2021,https://www.tripadvisor.com/Hotel_Review-g6071...,https://media-cdn.tripadvisor.com/media/photo-...,$$ (Based on Average Nightly Rates for a Stand...,5.0,962,562 Sutter St,San Francisco,United States,{},nan
1216,Staypineapple - An Elegant Hotel,"Steps from Union Square, and newly re-imagined...",Not the pineapple experience I had hoped for,None,NaN,January 2020,https://www.tripadvisor.com/Hotel_Review-g6071...,https://dynamic-media-cdn.tripadvisor.com/medi...,$ (Based on Average Nightly Rates for a Standa...,4.5,2016,580 Geary St,San Francisco,United States,{},nan
2043,San Francisco Marriott Fisherman's Wharf,Welcome to San Francisco Marriott Fisherman's ...,Great place to stay,The overall stay was a very pleasant one. The ...,5.0,October 2023,https://www.tripadvisor.com/Hotel_Review-g6071...,https://media-cdn.tripadvisor.com/media/photo-...,$$ (Based on Average Nightly Rates for a Stand...,4.0,1878,1250 Columbus Avenue,San Francisco,United States,{'room': {'sentence': ' The room we stayed in ...,hotel_name san francisco marriott fishermans w...
1931,Hyatt Place San Francisco Downtown,The Hyatt Place San Francisco Downtown is a LE...,"Great company, amazing service.",I have always been a fan of Hyatt but this Hya...,5.0,October 2023,https://www.tripadvisor.com/Hotel_Review-g6071...,https://media-cdn.tripadvisor.com/media/photo-...,$$ (Based on Average Nightly Rates for a Stand...,4.0,534,701 3rd St,San Francisco,United States,{},hotel_name hyatt place san francisco downtown ...
705,MySuite Istanbul,All residences offered by MySuite Istanbul are...,Great hotel,There was so much about this little hotel that...,5.0,May 2022,https://www.tripadvisor.com/Hotel_Review-g2939...,https://dynamic-media-cdn.tripadvisor.com/medi...,$ (Based on Average Nightly Rates for a Standa...,5.0,372,Firuzaga Mah. Turkgucu Caddesi No:74 / A,Istanbul,Türkiye,"{'room': {'sentence': 'The room was lovely, sm...",hotel_name mysuite istanbul city istanbul revi...


In [ ]:
sample['embedding'] = sample['reviews_search'].apply(lambda x: embedder.encode(x, convert_to_tensor=True))
sample.head()

,hotel_name,hotel_description,review_title,review_text,rate,tripdate,hotel_url,hotel_image,price_range,rating_value,review_count,street_address,locality,country,aspect_sentiments,reviews_search,embedding
1353,Hotel Emblem San Francisco,Check in and leave the status quo behind. Hote...,Wonderful Boutique Hotel!,None,NaN,November 2021,https://www.tripadvisor.com/Hotel_Review-g6071...,https://media-cdn.tripadvisor.com/media/photo-...,$$ (Based on Average Nightly Rates for a Stand...,5.0,962,562 Sutter St,San Francisco,United States,{},nan,"[tensor(0.2612), tensor(0.7683), tensor(-1.922..."
1216,Staypineapple - An Elegant Hotel,"Steps from Union Square, and newly re-imagined...",Not the pineapple experience I had hoped for,None,NaN,January 2020,https://www.tripadvisor.com/Hotel_Review-g6071...,https://dynamic-media-cdn.tripadvisor.com/medi...,$ (Based on Average Nightly Rates for a Standa...,4.5,2016,580 Geary St,San Francisco,United States,{},nan,"[tensor(0.2612), tensor(0.7683), tensor(-1.922..."
2043,San Francisco Marriott Fisherman's Wharf,Welcome to San Francisco Marriott Fisherman's ...,Great place to stay,The overall stay was a very pleasant one. The ...,5.0,October 2023,https://www.tripadvisor.com/Hotel_Review-g6071...,https://media-cdn.tripadvisor.com/media/photo-...,$$ (Based on Average Nightly Rates for a Stand...,4.0,1878,1250 Columbus Avenue,San Francisco,United States,{'room': {'sentence': ' The room we stayed in ...,hotel_name san francisco marriott fishermans w...,"[tensor(-0.9631), tensor(0.8613), tensor(-3.63..."
1931,Hyatt Place San Francisco Downtown,The Hyatt Place San Francisco Downtown is a LE...,"Great company, amazing service.",I have always been a fan of Hyatt but this Hya...,5.0,October 2023,https://www.tripadvisor.com/Hotel_Review-g6071...,https://media-cdn.tripadvisor.com/media/photo-...,$$ (Based on Average Nightly Rates for a Stand...,4.0,534,701 3rd St,San Francisco,United States,{},hotel_name hyatt place san francisco downtown ...,"[tensor(-0.7804), tensor(1.0249), tensor(-3.75..."
705,MySuite Istanbul,All residences offered by MySuite Istanbul are...,Great hotel,There was so much about this little hotel that...,5.0,May 2022,https://www.tripadvisor.com/Hotel_Review-g2939...,https://dynamic-media-cdn.tripadvisor.com/medi...,$ (Based on Average Nightly Rates for a Standa...,5.0,372,Firuzaga Mah. Turkgucu Caddesi No:74 / A,Istanbul,Türkiye,"{'room': {'sentence': 'The room was lovely, sm...",hotel_name mysuite istanbul city istanbul revi...,"[tensor(-2.0735), tensor(0.8177), tensor(-3.38..."


In [ ]:
sample.to_pickle('/content/drive/MyDrive/Task2/df.pkl')

In [ ]:
df_with_embedding = pd.read_pickle('/content/drive/MyDrive/Task2/df.pkl')

In [ ]:
df_with_embedding.head()

,hotel_name,hotel_description,review_title,review_text,rate,tripdate,hotel_url,hotel_image,price_range,rating_value,review_count,street_address,locality,country,aspect_sentiments,reviews_search,embedding
1353,Hotel Emblem San Francisco,Check in and leave the status quo behind. Hote...,Wonderful Boutique Hotel!,None,NaN,November 2021,https://www.tripadvisor.com/Hotel_Review-g6071...,https://media-cdn.tripadvisor.com/media/photo-...,$$ (Based on Average Nightly Rates for a Stand...,5.0,962,562 Sutter St,San Francisco,United States,{},nan,"[tensor(0.2612), tensor(0.7683), tensor(-1.922..."
1216,Staypineapple - An Elegant Hotel,"Steps from Union Square, and newly re-imagined...",Not the pineapple experience I had hoped for,None,NaN,January 2020,https://www.tripadvisor.com/Hotel_Review-g6071...,https://dynamic-media-cdn.tripadvisor.com/medi...,$ (Based on Average Nightly Rates for a Standa...,4.5,2016,580 Geary St,San Francisco,United States,{},nan,"[tensor(0.2612), tensor(0.7683), tensor(-1.922..."
2043,San Francisco Marriott Fisherman's Wharf,Welcome to San Francisco Marriott Fisherman's ...,Great place to stay,The overall stay was a very pleasant one. The ...,5.0,October 2023,https://www.tripadvisor.com/Hotel_Review-g6071...,https://media-cdn.tripadvisor.com/media/photo-...,$$ (Based on Average Nightly Rates for a Stand...,4.0,1878,1250 Columbus Avenue,San Francisco,United States,{'room': {'sentence': ' The room we stayed in ...,hotel_name san francisco marriott fishermans w...,"[tensor(-0.9631), tensor(0.8613), tensor(-3.63..."
1931,Hyatt Place San Francisco Downtown,The Hyatt Place San Francisco Downtown is a LE...,"Great company, amazing service.",I have always been a fan of Hyatt but this Hya...,5.0,October 2023,https://www.tripadvisor.com/Hotel_Review-g6071...,https://media-cdn.tripadvisor.com/media/photo-...,$$ (Based on Average Nightly Rates for a Stand...,4.0,534,701 3rd St,San Francisco,United States,{},hotel_name hyatt place san francisco downtown ...,"[tensor(-0.7804), tensor(1.0249), tensor(-3.75..."
705,MySuite Istanbul,All residences offered by MySuite Istanbul are...,Great hotel,There was so much about this little hotel that...,5.0,May 2022,https://www.tripadvisor.com/Hotel_Review-g2939...,https://dynamic-media-cdn.tripadvisor.com/medi...,$ (Based on Average Nightly Rates for a Standa...,5.0,372,Firuzaga Mah. Turkgucu Caddesi No:74 / A,Istanbul,Türkiye,"{'room': {'sentence': 'The room was lovely, sm...",hotel_name mysuite istanbul city istanbul revi...,"[tensor(-2.0735), tensor(0.8177), tensor(-3.38..."


In [ ]:
query = 'from paris i want best hotel'
query_embedding = embedder.encode(query,show_progress_bar=True)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
def search(query):

  n = 15

  query_embedding = embedder.encode(query)

  df_with_embedding["similarity"] = df_with_embedding.embedding.apply(lambda x: cosine_similarity(x, query_embedding.reshape(768,-1)))

  results = (
      df_with_embedding.sort_values("similarity", ascending=False)
      .head(n))
  resultlist = []

  hlist = []
  for r in results.index:
      if results.hotel_name[r] not in hlist:
          smalldf = results.loc[results.hotel_name == results.hotel_name[r]]
          if smalldf.shape[1] > 3:
            smalldf = smalldf[:3]
          resultlist.append(
          {
            "name":results.hotel_name[r],
            "score": smalldf.similarity[r][0],
            "rating": smalldf.rating_value.max(),
            "relevant_reviews": [ smalldf.review_text[s] for s in smalldf.index]
          })
          hlist.append(results.hotel_name[r])
  return resultlist

In [ ]:
df.head()

,hotel_name,hotel_description,review_title,review_text,rate,tripdate,hotel_url,hotel_image,price_range,rating_value,review_count,street_address,locality,country,aspect_sentiments,reviews_search
0,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,"An exceptional boutique hotel, great value for...",None,NaN,February 2020,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,$ (Based on Average Nightly Rates for a Standa...,5.0,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye,{},NaN
1,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,You can’t get better than this.,None,NaN,March 2021,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,$ (Based on Average Nightly Rates for a Standa...,5.0,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye,{},NaN
2,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,Exceeds all expectations,None,NaN,March 2021,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,$ (Based on Average Nightly Rates for a Standa...,5.0,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye,{},NaN
3,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,"Great Location, Fantastic Accommodations",None,NaN,August 2021,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,$ (Based on Average Nightly Rates for a Standa...,5.0,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye,{},NaN
4,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,Perfection. It is all in the details.,None,NaN,June 2021,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,$ (Based on Average Nightly Rates for a Standa...,5.0,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye,{},NaN


In [ ]:
search('a hotel in london and great food nearby but not too expensive')

[{'name': 'The Royal Horseguards',
  'score': 0.7111285,
  'rating': 4.5,
  'relevant_reviews': ['Hotel is lovely.  The highlight is the service.  Gonzalo snd his team in the restaurant and the best in London!  Absolutely perfect.  We will visit again because of Gonzolo and his team.  We have been so pampered and had delicious meals with impeccable service.  All hotels should have staff like this.  We will be back!']},
 {'name': 'The Resident Covent Garden',
  'score': 0.6928345,
  'rating': 5.0,
  'relevant_reviews': ['Fantastically clean, welcoming and great kitchenettes in each room.   You will not come across a more dedicated team in terms of customer service excellence.  Its our 3rd stay in 2 years and nothing changes  - I cannot fault this hotel and its team.  Great location, right in Covent Garden and The Strand, every type of coffee shop, cafe and restaurant on your doorstop. Chemist and supermarket close at hand. Great transport links with an easy 12 minute walk into Leicester

In [ ]:
def composite_score(embedding_similarity, aspect_sentiments, aspect_weights):
    sentiment_score = sum(
        aspect_weights.get(aspect, 1) * sentiment['score']
        for aspect, sentiment in aspect_sentiments.items()
    )
    sentiment_score /= len(aspect_sentiments) if aspect_sentiments else 1
    return embedding_similarity * 0.5 + sentiment_score * 0.5


aspect_weights = {"room": 1.2, "service": 1.0, "location": 1.0, "staff": 0.8, "food": 0.7, "noise": -1.0, "bed": 0.9, "view": 1.1}

In [ ]:
def search(query):
    n = 10

    query_embedding = embedder.encode(query)

    df["similarity"] = df['embedding'].apply(lambda x: cosine_similarity(x, query_embedding.reshape(768, -1)))

    def calculate_composite_score(row):
        aspect_sentiments = row['aspect_sentiments']
        embedding_similarity = row['similarity']
        return composite_score(embedding_similarity, aspect_sentiments, aspect_weights)

    df["composite_score"] = df.apply(calculate_composite_score, axis=1)

    results = df.sort_values("composite_score", ascending=False).head(n)

    resultlist = []
    hlist = []
    for r in results.index:
        if results.hotel_name[r] not in hlist:
            smalldf = results.loc[results.hotel_name == results.hotel_name[r]]
            if smalldf.shape[0] > 3:
                smalldf = smalldf.head(3)

            resultlist.append({
                "name": results.hotel_name[r],
                "score": smalldf.composite_score.mean(),
                "rating": smalldf.rating_value.max(),
                "relevant_reviews": [smalldf.review_text[s] for s in smalldf.index]
            })
            hlist.append(results.hotel_name[r])
    return resultlist

# **Entire Dataset**

In [ ]:
df_entire_dataset = df_copy.copy()

In [ ]:

df_entire_dataset['embedding'] = df_entire_dataset['reviews_search'].apply(lambda x: embedder.encode(x, convert_to_tensor=True))
df_entire_dataset.head()


,hotel_name,hotel_description,review_title,review_text,rate,tripdate,hotel_url,hotel_image,price_range,rating_value,review_count,street_address,locality,country,aspect_sentiments,reviews_search,embedding
0,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,"An exceptional boutique hotel, great value for...",None,NaN,February 2020,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,$ (Based on Average Nightly Rates for a Standa...,5.0,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye,{},nan,"[tensor(0.2612), tensor(0.7683), tensor(-1.922..."
1,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,You can’t get better than this.,None,NaN,March 2021,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,$ (Based on Average Nightly Rates for a Standa...,5.0,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye,{},nan,"[tensor(0.2612), tensor(0.7683), tensor(-1.922..."
2,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,Exceeds all expectations,None,NaN,March 2021,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,$ (Based on Average Nightly Rates for a Standa...,5.0,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye,{},nan,"[tensor(0.2612), tensor(0.7683), tensor(-1.922..."
3,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,"Great Location, Fantastic Accommodations",None,NaN,August 2021,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,$ (Based on Average Nightly Rates for a Standa...,5.0,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye,{},nan,"[tensor(0.2612), tensor(0.7683), tensor(-1.922..."
4,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,Perfection. It is all in the details.,None,NaN,June 2021,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,$ (Based on Average Nightly Rates for a Standa...,5.0,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye,{},nan,"[tensor(0.2612), tensor(0.7683), tensor(-1.922..."


In [ ]:
import gradio as gr
def search(query):
    n = 10
    query_embedding = embedder.encode(query)
    df_entire_dataset["similarity"] = df_entire_dataset['embedding'].apply(lambda x: cosine_similarity(x, query_embedding.reshape(768, -1)))
    def calculate_composite_score(row):
        aspect_sentiments = row['aspect_sentiments']
        embedding_similarity = row['similarity']
        return composite_score(embedding_similarity, aspect_sentiments, aspect_weights)
    df_entire_dataset["composite_score"] = df_entire_dataset.apply(calculate_composite_score, axis=1)
    results = df_entire_dataset.sort_values("composite_score", ascending=False).head(n)
    resultlist = []
    hlist = []
    for r in results.index:
        if results.hotel_name[r] not in hlist:
            smalldf = results.loc[results.hotel_name == results.hotel_name[r]]
            if smalldf.shape[0] > 3:
                smalldf = smalldf.head(3)
            resultlist.append({
                "name": results.hotel_name[r],
                "score": smalldf.composite_score.mean(),
                "rating": smalldf.rating_value.max(),
                "relevant_reviews": [smalldf.review_text[s] for s in smalldf.index]
            })
            hlist.append(results.hotel_name[r])
    return resultlist
def greet(query):
    bm25 = search(query)
    return bm25

demo = gr.Interface(fn=greet, inputs="text", outputs="json")

demo.launch(share=True,debug=False)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://5dbf1fc6c79d4c9977.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [1]:
!pip install openai==0.27.7
!pip install gradio
!pip install -U sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 801.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 435.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/1

In [2]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import json
import pandas as pd
import time
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from collections import Counter
from heapq import nlargest
import nltk
import numpy as np
from tqdm import tqdm
from sentence_transformers import SentenceTransformer, util
# import tiktoken
from openai.embeddings_utils import get_embedding, cosine_similarity

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [4]:
import json
from sentence_transformers import SentenceTransformer, CrossEncoder, util
import gzip
import os
import torch

embedder = SentenceTransformer('all-mpnet-base-v2')

# Use the GPU if available
if not torch.cuda.is_available():
    print("Warning: No GPU found. Please add GPU to your notebook")
else:
  print("GPU Found!")
  embedder =  embedder.to('cuda')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
import pandas as pd
df = pd.read_pickle('/content/drive/MyDrive/Task2/df.pkl')

In [6]:
df.shape

(100, 17)

In [7]:
df.head()

,hotel_name,hotel_description,review_title,review_text,rate,tripdate,hotel_url,hotel_image,price_range,rating_value,review_count,street_address,locality,country,aspect_sentiments,reviews_search,embedding
1353,Hotel Emblem San Francisco,Check in and leave the status quo behind. Hote...,Wonderful Boutique Hotel!,None,NaN,November 2021,https://www.tripadvisor.com/Hotel_Review-g6071...,https://media-cdn.tripadvisor.com/media/photo-...,$$ (Based on Average Nightly Rates for a Stand...,5.0,962,562 Sutter St,San Francisco,United States,{},nan,"[tensor(0.2612), tensor(0.7683), tensor(-1.922..."
1216,Staypineapple - An Elegant Hotel,"Steps from Union Square, and newly re-imagined...",Not the pineapple experience I had hoped for,None,NaN,January 2020,https://www.tripadvisor.com/Hotel_Review-g6071...,https://dynamic-media-cdn.tripadvisor.com/medi...,$ (Based on Average Nightly Rates for a Standa...,4.5,2016,580 Geary St,San Francisco,United States,{},nan,"[tensor(0.2612), tensor(0.7683), tensor(-1.922..."
2043,San Francisco Marriott Fisherman's Wharf,Welcome to San Francisco Marriott Fisherman's ...,Great place to stay,The overall stay was a very pleasant one. The ...,5.0,October 2023,https://www.tripadvisor.com/Hotel_Review-g6071...,https://media-cdn.tripadvisor.com/media/photo-...,$$ (Based on Average Nightly Rates for a Stand...,4.0,1878,1250 Columbus Avenue,San Francisco,United States,{'room': {'sentence': ' The room we stayed in ...,hotel_name san francisco marriott fishermans w...,"[tensor(-0.9631), tensor(0.8613), tensor(-3.63..."
1931,Hyatt Place San Francisco Downtown,The Hyatt Place San Francisco Downtown is a LE...,"Great company, amazing service.",I have always been a fan of Hyatt but this Hya...,5.0,October 2023,https://www.tripadvisor.com/Hotel_Review-g6071...,https://media-cdn.tripadvisor.com/media/photo-...,$$ (Based on Average Nightly Rates for a Stand...,4.0,534,701 3rd St,San Francisco,United States,{},hotel_name hyatt place san francisco downtown ...,"[tensor(-0.7804), tensor(1.0249), tensor(-3.75..."
705,MySuite Istanbul,All residences offered by MySuite Istanbul are...,Great hotel,There was so much about this little hotel that...,5.0,May 2022,https://www.tripadvisor.com/Hotel_Review-g2939...,https://dynamic-media-cdn.tripadvisor.com/medi...,$ (Based on Average Nightly Rates for a Standa...,5.0,372,Firuzaga Mah. Turkgucu Caddesi No:74 / A,Istanbul,Türkiye,"{'room': {'sentence': 'The room was lovely, sm...",hotel_name mysuite istanbul city istanbul revi...,"[tensor(-2.0735), tensor(0.8177), tensor(-3.38..."


In [12]:
def composite_score(embedding_similarity, aspect_sentiments, aspect_weights):
    sentiment_score = sum(
        aspect_weights.get(aspect, 1) * sentiment['score']
        for aspect, sentiment in aspect_sentiments.items()
    )
    sentiment_score /= len(aspect_sentiments) if aspect_sentiments else 1
    return embedding_similarity * 0.5 + sentiment_score * 0.5


aspect_weights = {"room": 1.2, "service": 1.0, "location": 1.0, "staff": 0.8, "food": 0.7, "noise": -1.0, "bed": 0.9, "view": 1.1}

In [13]:
def search(query):
    n = 10

    query_embedding = embedder.encode(query)

    df["similarity"] = df['embedding'].apply(lambda x: cosine_similarity(x, query_embedding.reshape(768, -1)))

    def calculate_composite_score(row):
        aspect_sentiments = row['aspect_sentiments']
        embedding_similarity = row['similarity']
        return composite_score(embedding_similarity, aspect_sentiments, aspect_weights)

    df["composite_score"] = df.apply(calculate_composite_score, axis=1)

    results = df.sort_values("composite_score", ascending=False).head(n)

    resultlist = []
    hlist = []
    for r in results.index:
        if results.hotel_name[r] not in hlist:
            smalldf = results.loc[results.hotel_name == results.hotel_name[r]]
            if smalldf.shape[0] > 3:
                smalldf = smalldf.head(3)

            resultlist.append({
                "name": results.hotel_name[r],
                "score": smalldf.composite_score.mean(),
                "rating": smalldf.rating_value.max(),
                "relevant_reviews": [smalldf.review_text[s] for s in smalldf.index]
            })
            hlist.append(results.hotel_name[r])
    return resultlist

In [14]:
query = 'hotel in london'

In [16]:
results = search(query)

In [17]:
results

[{'name': 'Margaritaville Resort Times Square',
  'score': array([0.65397525], dtype=float32),
  'rating': 5.0,
  'relevant_reviews': ["Johanna at the main desk check in was super helpful and friendly. She answered all our questions with a smile! The hotel was clean and everyone is so kind. Our room was small but we'll ladies out. We love the spacious shower. Overall a great NYC hotel room in the heart of theater  district"]},
 {'name': 'Sanctuary Hotel New York',
  'score': array([0.64791024], dtype=float32),
  'rating': 4.5,
  'relevant_reviews': ['Another expensive hotel booked with TUI and another disappointment! We came back at 6pm on different occasions to find our room had not been made up 😡 had to ring reception apologies made but not good enough when you’ve been out all day. Careful when you book rooms. TUI booked ours when we arrived the room was so small you had to walk around each other the wardrobe was a small hole in the wall. But the worst thing was THE SHOWER WAS ACTUAL

In [18]:
from google.colab import userdata
import openai
openai.api_key = userdata.get('OPENAI_API_KEY1')

In [19]:
def generate_answer(query):
    prompt = f"""
    Based on the following query from a user, please generate a detailed answer based on the context
    focusing on which is the top hotel based on the query. You should responsd as you are a  travel agent and are conversing with the
    user in a nice coordial way. Generate an answer as if you are a poet. Make sure to address the user as Khalid.
    remove the special characters and (/n ) , make the output clean and concise.

    ###########
    query:
    "{query}"

    ########

    context:"
    "{search(query)}"
    #####

    Return in Markdown format with each hotel highlighted.
    """

    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]
    response = openai.ChatCompletion.create(
        model="gpt-4o-mini",
        max_tokens=1500,
        n=1,
        stop=None,
        temperature=0.2, #higher temperature means more creative or more hallucination
        messages = messages


    )

    # Extract the generated response from the API response
    generated_text = response.choices[0].message['content'].strip()

    return generated_text

# # Example usage
# query = "What are the best amenities offered by Hotel XYZ?"
# response = generate_hotel_response(query)
# print(response)

    #return response.choices[0].message.content.strip()

In [20]:
txt=generate_answer('find hotel in london')

In [21]:
import markdown
from IPython.display import display, HTML

def render_markdown(md_text):
    # Convert Markdown to HTML
    html = markdown.markdown(md_text)
    # Display the HTML
    display(HTML(html))

In [22]:
render_markdown(txt)

In [23]:

import gradio as gr

# def search(query):
#   n = 15
#   query_embedding = embedder.encode(query)
#   df["similarity"] = df.embedding.apply(lambda x: cosine_similarity(x, query_embedding.reshape(768,-1)))

#   results = (
#       df.sort_values("similarity", ascending=False)
#       .head(n))

#   resultlist = []

#   hlist = []
#   for r in results.index:
#       if results.name[r] not in hlist:
#           smalldf = results.loc[results.name == results.name[r]]
#           smallarr = smalldf.similarity[r].max()
#           sm =smalldf.rating[r].mean()

#           if smalldf.shape[1] > 3:
#             smalldf = smalldf[:3]

#           resultlist.append(
#           {
#             "name":results.name[r],
#             "description":results.description[r],
#             "relevance score": smallarr.tolist(),
#             "rating": sm.tolist(),
#             "relevant_reviews": [ smalldf.text[s] for s in smalldf.index]
#           })
#           hlist.append(results.name[r])
#   return resultlist

def greet(query):
    bm25 = generate_answer(query)
    return bm25

# Use the gradio library to display a user interface for your user to interact with.
demo = gr.Interface(fn=greet, inputs="text", outputs="text")

# Launch the user demo - This can be done directly in your colab notebook. On your local notebook, you can also give a personalized localhost:port address.
demo.launch(share=True,debug=False)
# COPY THE URL THAT APPEARS BELOW

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://dd8a7031e1d741735a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
